In [ ]:
!pip install torch_snippets

In [ ]:
from torch_snippets import *
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from torchvision.utils import make_grid

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
tr_ds = datasets.MNIST(root = '/content/',train = True, transform = transforms.ToTensor(), download = True)
vl_ds = datasets.MNIST(root = '/content/',train = False, transform = transforms.ToTensor(), download = True)
tr_dl = DataLoader(tr_ds,batch_size = 64, shuffle = True, drop_last = True)
vl_dl = DataLoader(vl_ds,batch_size = 64, shuffle = True, drop_last = True)

In [ ]:
class VAE(nn.Module):
  def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
    super().__init__()
    self.d1 = nn.Linear(x_dim, h_dim1)
    self.d2 = nn.Linear(h_dim1, h_dim2)
    self.d31 = nn.Linear(h_dim2, z_dim)
    self.d32 = nn.Linear(h_dim2, z_dim)
    self.d4 = nn.Linear(z_dim, h_dim2)
    self.d5 = nn.Linear(h_dim2, h_dim1)
    self.d6 = nn.Linear(h_dim1, x_dim)
  
  def encoder(self, x):
    h = F.relu(self.d1(x))
    h = F.relu(self.d2(h))
    return self.d31(h), self.d32(h)
  
  def decoder(self, x):
    h = F.relu(self.d4(x))
    h = F.relu(self.d5(h))
    return F.sigmoid(self.d6(h))
  
  def sampling(self, mean, log_var):
    std = torch.exp(0.5*log_var)
    eps = torch.randn_like(std)
    return eps.mul(std).add_(mean)
  
  def forward(self, x):
    mean, log_var = self.encoder(x.view(-1, 784))
    z = self.sampling(mean, log_var)
    return self.decoder(z), mean, log_var

In [ ]:
def train_batch(data, model, optimizer, loss_function):
    model.train()
    data = data.to(device)
    optimizer.zero_grad()
    recon_batch, mean, log_var = model(data)
    loss, mse, kld = loss_function(recon_batch, data, mean, log_var)
    loss.backward()
    optimizer.step()
    return loss, mse, kld, log_var.mean(), mean.mean()

def validate_batch(data, model, loss_function):
    model.eval()
    data = data.to(device)
    recon, mean, log_var = model(data)
    loss, mse, kld = loss_function(recon, data, mean, log_var)
    return loss, mse, kld, log_var.mean(), mean.mean()
    
def loss_function(recon_x, x, mean, log_var):
    RECON = F.mse_loss(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    return RECON + KLD, RECON, KLD

In [ ]:
vae = VAE(x_dim=784, h_dim1=512, h_dim2=256, z_dim=50).to(device)
optimizer = optim.AdamW(vae.parameters(), lr=1e-3)
n_epochs = 5
log = Report(n_epochs)
for epoch in range(n_epochs):
    N = len(tr_dl)
    for batch_idx, (data, _) in enumerate(tr_dl):
        loss, recon, kld, log_var, mean = train_batch(data, vae, optimizer, loss_function)
        pos = epoch + (1+batch_idx)/N
        log.record(pos, train_loss=loss, train_kld=kld, train_recon=recon,train_log_var=log_var, train_mean=mean, end='\r')
    N = len(vl_dl)
    for batch_idx, (data, _) in enumerate(vl_dl):
        loss, recon, kld,log_var,mean = validate_batch(data, vae, loss_function)
        pos = epoch + (1+batch_idx)/N
        log.record(pos, val_loss=loss, val_kld=kld, val_recon=recon, val_log_var=log_var, val_mean=mean, end='\r')
    log.report_avgs(epoch+1)

log.plot_epochs(['train_loss','val_loss'])

In [ ]:
with torch.no_grad():
  z = torch.randn(64, 50).to(device)
  sample = vae.decoder(z).to(device)
  images = make_grid(sample.view(64, 1, 28, 28)).permute(1,2,0)
  show(images)